In [1]:
#Changing the working directory to the root
cd("c:/Users/ajaoo/Documents/GitHub/COVID-19-modelling")

println(pwd())


c:\Users\ajaoo\Documents\GitHub\COVID-19-modelling


In [2]:
using Distributions, StatsBase, CSV, DataFrames, StatsPlots, Plots, Dates, PlotThemes

In [3]:
# Set up Plots                                  
gr()
theme(:default)

In [4]:
function find_country(data_df, country::String)
    data_df_new = data_df[ismissing.(data_df[!, Symbol("Province/State")]), :]
    loc = findfirst(data_df_new[!, Symbol("Country/Region")] .== country)
    return data_df_new[loc, :]
end


find_country (generic function with 1 method)

In [5]:
function plot_timeseries(data, dates)
    @df data plot(dates, eachcol(data),
        label=names(data),
        xlabel="Time",
        ylabel="Total number of reported cases",
        xticks=dates[1:7:end],
        xrotation=45,
        marker=(:diamond, 4),
        line=(:line, "gray"),
        legend=:topleft,
        grid=false,
        framestyle=:semi,
        legendfontsize=9,
        tickfontsize=9,
        formatter=:plain
    )
end

function plot_daily_cases(data, dates)
    @df data bar(dates, eachcol(data),
        label=names(data),
        xlabel="Time",
        ylabel="Daily number of reported cases",
        xticks=dates[1:7:end],
        xrotation=45,
        legend=:topleft,
        grid=false,
        framestyle=:semi,
        legendfontsize=9,
        formatter=:plain
    )
end

plot_daily_cases (generic function with 1 method)

In [6]:
# Load data
data_path = "julia experiments/Data/covid_19_global_data.csv"
data_df = CSV.read(data_path, DataFrame)

Row,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,⋯
,String?,String,Float64?,Float64?,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,⋯
1,missing,Afghanistan,33.9391,67.71,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,5,5,5,5,5,5,5,5,5,5,5,8,8,8,8,11,11,11,14,20,25,26,26,26,24,24,34,40,42,74,80,91,106,114,114,166,192,235,269,270,299,337,367,423,444,521,521,555,607,665,770,794,845,908,933,996,1026,1092,1176,1226,1330,1463,⋯
2,missing,Albania,41.1533,20.1683,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,10,12,23,33,38,42,51,55,59,64,70,76,89,104,123,146,174,186,197,212,223,243,259,277,304,333,361,377,383,400,409,416,433,446,467,475,494,518,539,548,562,584,609,634,663,678,712,726,⋯
3,missing,Algeria,28.0339,1.6596,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,3,5,12,12,17,17,19,20,20,20,24,26,37,48,54,60,74,87,90,139,201,230,264,302,367,409,454,511,584,716,847,986,1171,1251,1320,1423,1468,1572,1666,1761,1825,1914,1983,2070,2160,2268,2418,2534,2629,2718,2811,2910,3007,3127,3256,3382,⋯
4,missing,Andorra,42.5063,1.5218,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,39,39,53,75,88,113,133,164,188,224,267,308,334,370,376,390,428,439,466,501,525,545,564,583,601,601,638,646,659,673,673,696,704,713,717,717,723,723,731,738,738,⋯
5,missing,Angola,-11.2027,17.8739,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2,3,3,3,4,4,5,7,7,7,8,8,8,10,14,16,17,19,19,19,19,19,19,19,19,19,19,24,24,24,24,25,25,25,25,26,⋯
6,missing,Antarctica,-71.9499,23.347,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
7,missing,Antigua and Barbuda,17.0608,-61.7964,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,3,3,3,7,7,7,7,7,7,7,9,15,15,15,15,19,19,19,19,21,21,23,23,23,23,23,23,23,23,23,24,24,24,24,24,⋯
8,missing,Argentina,-38.4161,-63.6167,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,2,8,12,12,17,19,19,31,34,45,56,68,79,97,128,158,266,301,387,387,502,589,690,745,820,1054,1054,1133,1265,1451,1451,1554,1628,1715,1795,1975,1975,2142,2208,2277,2443,2571,2669,2758,2839,2941,3031,3144,3435,3607,3780,3892,⋯
9,missing,Armenia,40.0691,45.0382,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,4,8,18,26,52,78,84,115,136,160,194,235,249,265,290,329,407,424,482,532,571,663,736,770,822,833,853,881,9

In [7]:
# Process time series data for multiple countries
date_strings = names(data_df)[5:end]
format = Dates.DateFormat("m/d/y")
x = parse.(Date, date_strings, format) .+ Year(2000)

1143-element Vector{Date}:
 2020-01-22
 2020-01-23
 2020-01-24
 2020-01-25
 2020-01-26
 2020-01-27
 2020-01-28
 2020-01-29
 2020-01-30
 2020-01-31
 ⋮
 2023-03-01
 2023-03-02
 2023-03-03
 2023-03-04
 2023-03-05
 2023-03-06
 2023-03-07
 2023-03-08
 2023-03-09

In [8]:
countries = ["US", "Russia", "Italy", "Iran"]
y = DataFrame()

0×0 DataFrame

In [10]:
for country in countries
    country_data = find_country(data_df, country)
    stacked_data = stack(DataFrame(country_data), 5:length(country_data))
    y[!, Symbol(country)] = stacked_data[!, :value]
end

if "India" in names(y)
    y[!, :India][141] = 286131
end
y = y[1:150, :]

Row,US,Russia,Italy,Iran
,Int64,Int64,Int64,Int64
1,1,0,0,0
2,1,0,0,0
3,2,0,0,0
4,2,0,0,0
5,5,0,0,0
6,5,0,0,0
7,5,0,0,0
8,6,0,0,0
9,6,0,0,0


In [11]:
# Plotting
plot_timeseries(y, x[1:150])

ErrorException: Cannot convert DataFrames.DataFrameColumns{DataFrame} to series data for plotting

In [6]:
date_strings = names(data_df)[5:end]
format = Dates.DateFormat("m/d/y")
x = parse.(Date, date_strings, format) .+ Year(2000)

countries = ["US", "Russia", "Italy", "Iran"]
y = DataFrame()

0×0 DataFrame

In [9]:
for country in countries

    data_dfr = find_country(data_df, country) # returns a dataframe row 
    data_dfr = DataFrame(data_dfr)           # convert dataframe row back to a dataframe
    rows, cols = size(data_dfr)
    data_dfl = stack(data_dfr, 5:cols)       # convert dataframe into long format
    y[!, Symbol("$country")] = data_dfl[!, :value]
end

if "India" in names(y)
    y[!, :India][141] = 286131 # correction for number of cases reported on 10-06-2020
end

y = y[1:150, :]

Row,US,Russia,Italy,Iran
,Int64,Int64,Int64,Int64
1,1,0,0,0
2,1,0,0,0
3,2,0,0,0
4,2,0,0,0
5,5,0,0,0
6,5,0,0,0
7,5,0,0,0
8,6,0,0,0
9,6,0,0,0


In [10]:
# Plot time series data
plot_settings = (size=(900, 600), xlabel="Time", ylabel="Total number of reported cases", xticks=x[1:7:end], xrotation=45, legend=:topleft, grid=false, framestyle=:semi, legendfontsize=9, tickfontsize=9, formatter=:plain)
@df y plot(x[1:150], eachcol(y), label=names(y), marker=(:diamond, 4), line=(:line, "gray"), plot_settings...)
y.One_million = fill(10^6, size(y, 1))
display(@df y plot!(x, y[!, :One_million], linestyle=:dot, linewidth=5, color=:red, label="One_million"))
savefig("julia experiments/figures/Time_series_1.png")

ErrorException: Couldn't process recipe args: (Formatted{Vector{Int64}}, DataFrames.DataFrameColumns{DataFrame}, Int64, String, String, Formatted{Vector{Int64}}, Int64, Symbol, Bool, Symbol, Int64, Int64, Symbol)